## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

import sys
sys.path.append('../')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kapaa,US,22.08,-159.32,82.40,65,1,4.70,clear sky
1,1,Hervey Bay,AU,-25.30,152.85,72.00,93,81,4.00,broken clouds
2,2,Cape Town,ZA,-33.93,18.42,64.00,67,0,3.36,clear sky
3,3,Bubaque,GW,11.28,-15.83,80.83,62,0,10.40,clear sky
4,4,Port Alfred,ZA,-33.59,26.89,66.00,96,25,18.01,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"]>= min_temp) & (city_data_df["Max Temp"]<= max_temp)]
filtered_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hervey Bay,AU,-25.30,152.85,72.00,93,81,4.00,broken clouds
2,2,Cape Town,ZA,-33.93,18.42,64.00,67,0,3.36,clear sky
4,4,Port Alfred,ZA,-33.59,26.89,66.00,96,25,18.01,scattered clouds
11,11,Los Llanos De Aridane,ES,28.66,-17.92,66.20,82,90,2.24,overcast clouds
13,13,Katsina,NG,12.99,7.60,72.57,21,0,4.23,clear sky
19,19,New Norfolk,AU,-42.78,147.06,71.01,54,0,3.00,clear sky
21,21,Hobart,AU,-42.88,147.33,71.01,49,20,14.99,few clouds
23,23,Maine-Soroa,NE,13.21,12.02,74.50,19,0,6.89,clear sky
26,26,Ballina,AU,-28.87,153.57,71.60,83,90,27.51,overcast clouds
27,27,Lompoc,US,34.64,-120.46,62.01,82,1,5.82,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
filtered_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = filtered_cities_df.dropna()
clean_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hervey Bay,AU,-25.30,152.85,72.00,93,81,4.00,broken clouds
2,2,Cape Town,ZA,-33.93,18.42,64.00,67,0,3.36,clear sky
4,4,Port Alfred,ZA,-33.59,26.89,66.00,96,25,18.01,scattered clouds
11,11,Los Llanos De Aridane,ES,28.66,-17.92,66.20,82,90,2.24,overcast clouds
13,13,Katsina,NG,12.99,7.60,72.57,21,0,4.23,clear sky
...,...,...,...,...,...,...,...,...,...,...
643,643,Adrar,MR,20.50,-10.07,66.38,25,100,7.54,overcast clouds
657,657,Conceicao Da Barra,BR,-18.59,-39.73,74.71,79,7,7.05,clear sky
662,662,Januaria,BR,-15.48,-44.37,70.16,58,0,2.01,clear sky
663,663,Cowra,AU,-33.83,148.68,62.01,74,0,5.01,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hervey Bay,AU,72.00,broken clouds,-25.30,152.85,
2,Cape Town,ZA,64.00,clear sky,-33.93,18.42,
4,Port Alfred,ZA,66.00,scattered clouds,-33.59,26.89,
11,Los Llanos De Aridane,ES,66.20,overcast clouds,28.66,-17.92,
13,Katsina,NG,72.57,clear sky,12.99,7.60,
19,New Norfolk,AU,71.01,clear sky,-42.78,147.06,
21,Hobart,AU,71.01,few clouds,-42.88,147.33,
23,Maine-Soroa,NE,74.50,clear sky,13.21,12.02,
26,Ballina,AU,71.60,overcast clouds,-28.87,153.57,
27,Lompoc,US,62.01,clear sky,34.64,-120.46,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"] = hotel_df["Hotel Name"].replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(30)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hervey Bay,AU,72.00,broken clouds,-25.30,152.85,Shelly Bay Resort
2,Cape Town,ZA,64.00,clear sky,-33.93,18.42,Southern Sun Waterfront Cape Town
4,Port Alfred,ZA,66.00,scattered clouds,-33.59,26.89,The Halyards Hotel
11,Los Llanos De Aridane,ES,66.20,overcast clouds,28.66,-17.92,Valle Aridane
13,Katsina,NG,72.57,clear sky,12.99,7.60,Katsina Motel
19,New Norfolk,AU,71.01,clear sky,-42.78,147.06,The Shingles Riverside Cottages
21,Hobart,AU,71.01,few clouds,-42.88,147.33,St Ives Hobart Accommodation Tasmania
26,Ballina,AU,71.60,overcast clouds,-28.87,153.57,Ramada Hotel & Suites by Wyndham Ballina Byron
27,Lompoc,US,62.01,clear sky,34.64,-120.46,Holiday Inn Express Lompoc
28,Chuy,UY,69.31,overcast clouds,-33.70,-53.46,Nuevo Hotel Plaza


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")


In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=max_temp,
                                 dissipating=False,
                                 max_intensity=300, 
                                 point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))